# Metrics
This notebook will dive deep into metrics used to determine nudging.

In [ ]:
import sys
import os
from pathlib import Path

# go to project root
project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from nudging.models import OllamaClient
from nudging.data_loader import load_data
from nudging.experiment import run_single_experiment
import configs.experiment_config as experiment_config

import pandas as pd

#config = experiment_config.EXPERIMENT_BASELINE
config = experiment_config.EXPERIMENT_EXTENDED
print(f"Running experiment: {config.name}")
print(f"Context percentages: {config.context_percentages}")

# initialise the client
client = OllamaClient(model=config.model_config.name)
print(f"Model initialised: {config.model_config.name}")

# load data
dataset = load_data(
    base_dir=project_root / config.data_config.data_folder_name,
    min_words=config.data_config.min_word_count,
    max_samples=config.max_samples,
    categories=config.data_config.categories
)
print(f"loaded the data: {len(dataset)} files.")

Running experiment: memorisation_extended
Context percentages: [5, 25, 50, 75, 90]
Model initialised: qwen3:0.6b


AttributeError: module 'configs.experiment_config' has no attribute 'data_config'